# This notebook needs to be ran in Google Colab

In [ ]:
import warnings

# Set a global filter to ignore the specific DeprecationWarning from jupyter_client
# This is just needed cause something was happening in my Colab notebook where a deprecation message was popping up
warnings.filterwarnings(
    "ignore",
    message="datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).",
    category=DeprecationWarning,
    module="jupyter_client"
)

In [ ]:
!pip install --quiet --upgrade \
  google-colab==1.0.0 \
  requests==2.32.4 \
  langchain==0.3.27 \
  langchain-core==0.3.79 \
  huggingface_hub \
  langchain_huggingface \
  langchain-community \

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Prompting without Langchain - still using some Langchain packages just cause why not

from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

question = input("What is your question? ")

template = "{question}"

prompt = PromptTemplate(template=template, input_variables=["question"])

model_id = "Qwen/Qwen2.5-1.5B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    max_new_tokens=256,
)

llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = prompt | llm

response = llm_chain.invoke({"question": question})

print(response)

In [ ]:
# We run this seperately just in case - probably isnt needed
pip install transformers -U

In [ ]:
# Using Langchain to search the webs

from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.tools import BraveSearch
from langchain_core.runnables import RunnableLambda

question = input("What is your question? ")

# Replace with your actual Brave API key
brave_api = "BRAVE_API_KEY_HERE"

search = BraveSearch.from_api_key(api_key=brave_api, search_kwargs={"count": 5})
def fetch_results(q):
  return {"question":q, "search_results": search.run(q)}

# Template that is fed into the LLM model
template = """Use the web search results to answer the question.
Cite sources inline by title if helpful.

Question: {question}

Web search results:
{search_results}

Answer:"""

prompt = PromptTemplate(template=template)

model_id = "Qwen/Qwen2.5-1.5B-Instruct"
pipe = pipeline("text-generation", model=model_id, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

llm_chain = RunnableLambda(fetch_results) | prompt | llm
print(llm_chain.invoke(question))